In [1]:
import cv2
import numpy as np
import os

# 10.1

In [3]:
# Funkcja merge_figures łączy prostokąty należące do jednej sylwetki,
# które zostały wykryte jako oddzielne obiekty (np. tułów i nogi).
# Łączenie opiera się na bliskości w pionie i poziomie (th_distance).
# Prostokąty są scalane, jeśli są wystarczająco blisko siebie,
# a wynikowy obszar przekracza zadany próg min_area.

def merge_figures(stats, th_distance=25, min_area=1000):
    merged = []
    skip_indices = set()
    for i in range(len(stats)):
        if i in skip_indices:
            continue
        x, y, w, h, area = stats[i]
        aspect_ratio = h / w if w != 0 else 0
        if aspect_ratio > 0.6 and area > 500:
            current_rect = [x, y, x + w, y + h]
            for j in range(i + 1, len(stats)):
                if j in skip_indices:
                    continue
                x2, y2, w2, h2, area2 = stats[j]

                vert_proximity = abs((y + h) - y2) < th_distance or abs((y2 + h2) - y) < th_distance
                hor_proximity = abs(x - x2) < th_distance or abs((x + w) - (x2 + w2)) < th_distance

                if vert_proximity and hor_proximity:
                    new_x = min(current_rect[0], x2)
                    new_y = min(current_rect[1], y2)
                    new_x2 = max(current_rect[2], x2 + w2)
                    new_y2 = max(current_rect[3], y2 + h2)
                    current_rect = [new_x, new_y, new_x2, new_y2]
                    skip_indices.add(j)

            total_area = (current_rect[2] - current_rect[0]) * (current_rect[3] - current_rect[1])
            if total_area > min_area:
                merged.append(current_rect)
    return merged


cap = cv2.VideoCapture('vid1_IR.wm')
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

out = cv2.VideoWriter('output_IR_merged.avi',
                      cv2.VideoWriter_fourcc(*'XVID'),
                      fps,
                      (frame_width, frame_height))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    G = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    G_bin = cv2.threshold(G, 40, 255, cv2.THRESH_BINARY)[1]
    G_bin = cv2.medianBlur(G_bin, 5)
    kernel = np.ones((5, 5), np.uint8)
    G_bin = cv2.morphologyEx(G_bin, cv2.MORPH_CLOSE, kernel)

    num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(G_bin)

    merged_rects = merge_figures(stats[1:])

    output = frame.copy()
    for x1, y1, x2, y2 in merged_rects:
        cv2.rectangle(output, (x1, y1), (x2, y2), (0, 255, 0), 2)

    out.write(output)
    cv2.imshow('Output', output)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
out.release()
cv2.destroyAllWindows()